# Génération d’un patron de point de croix à partir d’une image

Ce notebook a pour objectif de transformer une image en **patron de point de croix**. Il va :
1. Permettre l’upload d’une image via un widget.
2. Envoyer cette image (ou un prompt associé) à l’API d’un service **Stable Diffusion** pour en obtenir une version “pixel art”.
3. Réduire la palette de couleurs et associer chaque couleur aux fils **DMC** les plus proches en s’appuyant sur un fichier JSON local.
4. Générer une grille adaptée, avec un symbole distinct pour chaque couleur, afin de faciliter la broderie.

Les étapes suivantes vous guideront pas à pas :
- **Installation des dépendances**  
- **Imports et configuration**  
- **Upload de l’image**  
- … *(à compléter ensuite)* …


## Installation / Mise à jour des dépendances

La cellule suivante permet d’installer ou de mettre à jour les librairies nécessaires :

- `numpy` pour le traitement en tableaux numériques.
- `pillow` pour la gestion des images (PIL).
- `requests` pour les requêtes HTTP (appel à l’API).
- `ipywidgets` pour les widgets interactifs.
- `jupyter-ui-poll` pour la boucle d’événements asynchrone dans Jupyter.


In [ ]:
# Installe/Met à jour les bibliothèques manquantes.

%pip install numpy pillow requests ipywidgets jupyter-ui-poll --quiet


In [ ]:
import requests
import base64
import json
import io

from PIL import Image
from IPython.display import display
import ipywidgets as widgets

# Chargement du fichier local DMC_colors.json
# (Assurez-vous que le fichier DMC_colors.json est bien dans le même dossier que le notebook)
try:
    with open("DMC_colors.json", "r", encoding="utf-8") as f:
        DMC_COLORS = json.load(f)
    print(f"Nombre de références DMC chargées : {len(DMC_COLORS)}")
except FileNotFoundError:
    DMC_COLORS = []
    print("Fichier DMC_colors.json non trouvé. Veuillez vérifier son emplacement.")
    


## Paramètres de génération (img2img)

- **prompt** : C’est la description textuelle qui oriente la génération.  
- **steps** : Nombre d’itérations de sampling (plus haut = plus de détails, plus long).  
- **sampler_name** : Algorithme de sampling (ex. Euler a, DPM++ 2M, etc.).  
- **cfg_scale** : Facteur de guidance (plus c’est élevé, plus l’image colle au prompt, mais peut être moins créative).  
- **width / height** : Dimensions de l’image en pixels.  
- **denoising_strength** : Dans le contexte img2img, indique dans quelle mesure l’image initiale est altérée par la diffusion (0.0 = pas de changement, 1.0 = radicalement transformée).  
- **n_iter** / **batch_size** : Contrôlent le nombre d’images générées. Ici, nous renvoyons 3 images pour laisser le choix.  


In [ ]:
from dotenv import load_dotenv
import os

# Paramètre de base pour l’endpoint Stable Diffusion

# Chargement des variables d'environnement depuis .env
load_dotenv()

# Récupération de l'URL de l'API Stable Diffusion depuis .env
SD_BASE_URL = os.getenv("SD_BASE_URL", "https://stable-diffusion-webui-forge.yourdomain.com")

# Construct the full API URL by appending the endpoint path
SD_API_URL = f"{SD_BASE_URL}/sdapi/v1/img2img"

# Paramètres par défaut pour la requête (vous pouvez les affiner à la demande)
default_img2img_payload = {
    "prompt": "<lora:pixelbuildings128-v2:1> Pixel Art",
    "steps": 20,
    "sampler_name": "DPM++ 2M SDE",
    "scheduler": "karras",
    "cfg_scale": 7.5,
    "width": 1024,
    "height": 1024,
    "denoising_strength": 0.37,
    "override_settings": {
        "sd_model_checkpoint": "sd_xl_base_1.0"
    },
    # "seed": 2096377536
}


### Upload de l’image

Dans la prochaine cellule, vous pourrez sélectionner l’image que vous souhaitez transformer en patron de point de croix.

- Choisissez un fichier **.png**, **.jpg**, ou **.jpeg** depuis votre ordinateur.
- Le fichier sera ensuite stocké en mémoire dans le notebook (nous le lirons pour l’envoyer à l’API, ou pour l’afficher).

**Étapes suivantes** (à venir) :
- Nous enverrons l’image à notre endpoint Stable Diffusion pour obtenir la version "pixel art".
- Nous appliquerons ensuite une réduction de palette et un matching avec les codes DMC.
- Enfin, nous générerons la grille de points de croix.


In [ ]:
import time
from jupyter_ui_poll import ui_events

# ========================
# Widgets pour l'upload et la génération
# ========================
upload_btn = widgets.FileUpload(
    accept='image/*',
    multiple=False,
    description='Uploader une image'
)

generate_btn = widgets.Button(
    description="Générer (img2img, 3 variantes)",
    button_style='success',
    icon='magic'
)

upload_output = widgets.Output()
generate_output = widgets.Output()

# ========================
# Widgets pour la sélection de l'image préférée
# ========================
selection_dropdown = widgets.Dropdown(
    options=[],
    description='Choix :',
    disabled=False
)
confirm_selection_btn = widgets.Button(
    description="Valider ce choix",
    button_style='info',
    icon='check'
)
selection_output = widgets.Output()

# ========================
# Variables de contrôle
# ========================
image_uploaded = False
generation_done = False
encoded_init_image = None
candidate_images = []  # contiendra la liste des images générées (PIL)

# ========================
# Callbacks
# ========================
def on_upload_change(change):
    """
    Callback appelé lorsque l'utilisateur a uploadé un fichier.
    """
    global image_uploaded, encoded_init_image
    if upload_btn.value:
        file_info = upload_btn.value[0]
        file_bytes = file_info['content']

        # Convertir en base64 pour l'envoyer à l'API
        encoded_init_image = base64.b64encode(file_bytes).decode('utf-8')

        with upload_output:
            upload_output.clear_output()
            print("✅ Image chargée. Taille (octets) :", len(file_bytes))
            # Affichage de l'image en miniature
            img = Image.open(io.BytesIO(file_bytes))
            display(img)

        image_uploaded = True

def on_generate_click_batch(button):
    """
    Callback appelé lorsque l'utilisateur clique sur 'Générer'.
    Effectue l'appel à l'API /sdapi/v1/img2img pour générer 3 images.
    """
    global generation_done, candidate_images
    generation_done = False
    candidate_images = []

    if not image_uploaded or not encoded_init_image:
        with generate_output:
            generate_output.clear_output()
            print("❌ Veuillez d'abord uploader une image.")
        return

    # Construire le payload pour l'img2img, en générant 3 images
    payload = default_img2img_payload.copy()
    payload["init_images"] = [encoded_init_image]
    payload["n_iter"] = 1
    payload["batch_size"] = 3

    with generate_output:
        generate_output.clear_output()
        print("⏳ Génération en cours (3 images)...")

        try:
            response = requests.post(url=SD_API_URL, json=payload, timeout=120)
            r = response.json()

            # Récupérer la liste des images générées (base64)
            result_images = r.get('images', [])
            if not result_images:
                print("❌ Aucune image reçue de l'API.")
            else:
                print(f"✅ {len(result_images)} image(s) générée(s).")
                for idx, img_b64 in enumerate(result_images):
                    img_data = base64.b64decode(img_b64)
                    img_pil = Image.open(io.BytesIO(img_data))

                    # On stocke l'image dans la liste
                    candidate_images.append(img_pil)

                    # Affichage rapide
                    display(img_pil)
                    print(f"↑ Image #{idx} ↑\n")

                # On remplit le dropdown pour la sélection
                selection_dropdown.options = [
                    (f"Image #{i}", i) for i in range(len(candidate_images))
                ]
                selection_dropdown.value = 0  # sélection par défaut de la première
                print("👉 Choisissez votre image préférée ci-dessous, puis cliquez sur 'Valider ce choix'.")

            generation_done = True

        except Exception as e:
            print(f"❌ Erreur lors de l'appel à l'API img2img : {e}")

def on_confirm_selection_click(b):
    """
    Callback pour valider l'image sélectionnée dans le dropdown.
    """
    global generated_image  # On mettra l'image choisie ici
    with selection_output:
        selection_output.clear_output()

        if not candidate_images:
            print("❌ Aucune image à sélectionner. Veuillez d'abord générer.")
            return

        chosen_idx = selection_dropdown.value
        chosen_img = candidate_images[chosen_idx]
        generated_image = chosen_img  # On définit l'image globale

        print(f"✅ Vous avez choisi l'image #{chosen_idx}. Elle est maintenant stockée dans 'generated_image'.")

# ========================
# Liaisons des callbacks
# ========================
upload_btn.observe(on_upload_change, names='value')
generate_btn.on_click(on_generate_click_batch)
confirm_selection_btn.on_click(on_confirm_selection_click)

# ========================
# Affichage des widgets
# ========================
display(widgets.HTML("<h3>Étape : Chargement de l'image et génération (3 variantes)</h3>"))
display(upload_btn)
display(upload_output)
display(generate_btn)
display(generate_output)

display(widgets.HTML("<h4>Étape : Sélection de l'image préférée</h4>"))
display(selection_dropdown)
display(confirm_selection_btn)
display(selection_output)

# ========================
# Boucle bloquante (optionnelle)
# ========================
print("En attente de l'upload puis du clic sur 'Générer (img2img, 3 variantes)' ...")

with ui_events() as poll:
    # On attend que la génération soit terminée
    while not generation_done:
        # IMPORTANT : poll() ne doit pas recevoir un float, mais un entier.
        poll(1)
        time.sleep(0.1)

print("✅ Génération terminée ! Vous pouvez passer à la suite (ou refaire une génération).")


### Réduction de l’image en blocs de 8×8

Le modèle de diffusion nous donne une image de 1024×800 pixels, mais il s’agit visuellement d’un « pixel art » où chaque “carré” mesure 8×8 pixels.  
Pour simplifier le traitement, nous allons réduire l’image de la façon suivante :

1. **Parcourir** l’image d’entrée par blocs de 8×8.  
2. **Calculer la couleur moyenne** (ou dominante) de chaque bloc.  
3. **Construire** une nouvelle image dont la taille sera (largeur/8) × (hauteur/8).  
4. Chaque pixel de cette nouvelle image représentera le bloc 8×8 original.

Ce résultat final de 128×100 (si l’image initiale fait 1024×800) sera bien plus simple à manipuler pour la suite du matching couleurs (DMC) et la création de la grille finale.


In [ ]:
import numpy as np

BLOCK_SIZE = 8

def reduce_by_blocks(img: Image.Image, block_size: int = 8) -> Image.Image:
    """
    Réduit l'image en regroupant chaque bloc block_size x block_size
    en un seul pixel, basé sur la couleur moyenne de ce bloc.
    """
    # Convertir l'image en tableau numpy (RGBA ou RGB)
    arr = np.array(img.convert("RGB"))
    h, w, _ = arr.shape
    
    # Dimensions du résultat final
    new_w = w // block_size
    new_h = h // block_size
    
    # Tableau numpy pour accueillir le résultat
    small_arr = np.zeros((new_h, new_w, 3), dtype=np.uint8)
    
    # Parcours par blocs
    for y in range(new_h):
        for x in range(new_w):
            # Coordonnées du bloc
            y0 = y * block_size
            x0 = x * block_size
            
            # Sous-tableau correspondant au bloc (block_size x block_size x 3)
            block = arr[y0:y0+block_size, x0:x0+block_size, :]
            
            # Moyenne sur l'axe (0,1) => (hauteur, largeur du bloc)
            mean_color = block.mean(axis=(0,1))
            
            # On assigne au pixel (y, x)
            small_arr[y, x] = mean_color.astype(np.uint8)
    
    # Création de l'image PIL à partir du tableau small_arr
    small_img = Image.fromarray(small_arr, mode="RGB")
    return small_img

# Exemple d'utilisation
# On suppose que vous avez déjà une variable `generated_image` (la sortie stable diffusion).
# Si vous avez besoin de la relire depuis le disque, faites : generated_image = Image.open("nom_fichier.png")

if 'generated_image' in globals():
    reduced_image = reduce_by_blocks(generated_image, block_size=BLOCK_SIZE)
    
    # Affichage du résultat
    display(reduced_image)
    print(f"Nouvelle dimension : {reduced_image.width} x {reduced_image.height}")
else:
    print("⚠️ La variable 'generated_image' n'est pas définie. Veuillez définir ou charger l'image d'abord.")


### Matching des couleurs et conversion vers la palette DMC

Maintenant que nous avons une **image réduite** (par blocs 8×8) avec une dimension plus raisonnable (p. ex. 128×100), nous allons :

1. **Analyser chaque pixel** pour connaître sa couleur.  
2. Pour chacun de ces pixels (R, G, B), **trouver la couleur de fil DMC la plus proche**. Nous utiliserons un fichier `DMC_colors.json` contenant l’équivalence entre un code de fil (ex. 310, 498...) et un triplet (R, G, B).  
3. Cette étape nous donnera une image où chaque pixel est remplacé par la couleur “officielle” du fil DMC correspondant, et/ou un tableau d’index de couleurs.  
4. Enfin, nous pourrons générer la **grille de point de croix**, où chaque couleur DMC aura un **symbole** distinct pour la lisibilité.


In [ ]:
import math
import numpy as np
from PIL import Image

def get_closest_dmc_color(r, g, b, dmc_list):
    """
    Retourne (floss_code, floss_name, (R_dmc, G_dmc, B_dmc)) de la couleur DMC 
    la plus proche de (r,g,b). floss_code est une chaîne (car parfois c'est "White", "B5200"...).
    """
    best_dist = float('inf')
    best_data = ("UNKNOWN", "NoName", (0, 0, 0))

    # Convertir r, g, b en int "classique" de Python (pas numpy)
    r = int(r)
    g = int(g)
    b = int(b)

    for item in dmc_list:
        try:
            R_dmc = int(item["r"])
            G_dmc = int(item["g"])
            B_dmc = int(item["b"])
        except (ValueError, TypeError, KeyError):
            # Si on ne peut pas convertir en entier, on saute
            continue

        # Calcul de la distance au carré en Python pur
        d_r = R_dmc - r
        d_g = G_dmc - g
        d_b = B_dmc - b
        dist = d_r*d_r + d_g*d_g + d_b*d_b

        if dist < best_dist:
            best_dist = dist
            floss_code = str(item["floss"])  # ex: "White", "B5200", "310"
            floss_name = item.get("name", "Unknown")
            best_data = (floss_code, floss_name, (R_dmc, G_dmc, B_dmc))

    return best_data


def convert_image_to_dmc(img: Image.Image, dmc_list):
    """
    Parcourt chaque pixel de l'image (img) et renvoie:
      - new_img: PIL Image recolorisée avec la couleur DMC la plus proche
      - dmc_codes: tableau 2D (h,w) des codes 'floss' (str)
    """
    # On convertit l'image en 'RGB', puis en numpy array
    arr_rgb = np.array(img.convert("RGB"), dtype=np.int16)
    h, w, _ = arr_rgb.shape

    # Création de deux tableaux de sortie
    dmc_arr   = np.zeros((h, w, 3), dtype=np.uint8)   # image DMC (R,G,B)
    dmc_codes = np.empty((h, w), dtype=object)        # codes DMC (str)

    for y in range(h):
        for x in range(w):
            # On récupère la couleur du pixel en int16 (0..255 mais sign-safe)
            r, g, b = arr_rgb[y, x]
            floss_code, floss_name, (dr, dg, db) = get_closest_dmc_color(r, g, b, dmc_list)

            dmc_arr[y, x]   = [dr, dg, db]
            dmc_codes[y, x] = floss_code

    # On convertit dmc_arr en image PIL
    new_img = Image.fromarray(dmc_arr, mode="RGB")
    return new_img, dmc_codes


# Exemple d’utilisation
if 'reduced_image' in globals():
    dmc_image, dmc_codes_array = convert_image_to_dmc(reduced_image, DMC_COLORS)
    display(dmc_image)
    print("Exemple: code DMC du pixel (0,0) =", dmc_codes_array[0,0])
else:
    print("⚠️ 'reduced_image' n'est pas défini.")
